In [ ]:
from pydantic import BaseModel, EmailStr, HttpUrl, ValidationError
from pydantic_extra_types.phone_numbers import PhoneNumber
from typing import List, Optional
from openai import OpenAI
import instructor
from dotenv import load_dotenv
import docx2txt
import os
import json
import logging
from fastapi import FastAPI, File, UploadFile #type: ignore
from fastapi.responses import JSONResponse #type: ignore
import timeit

load_dotenv() 

app = FastAPI()

SYSTEM_PROMPT = """
You are a professional resume parser.

Your task is to extract and structure information from an unstructured resume text according to the JSON schema provided separately in the system or developer context.

Follow these rules:

1. Read and interpret the resume text carefully.
2. Extract the following information and populate the corresponding fields in the schema:
   - Name
   - Email
   - Phone number
   - URLs (LinkedIn, portfolio, or personal websites)
   - Professional summary - This contains a list of bullet points
   - Technical skills
   - Education details (degree, institution, location, start and end years if available)
   - Professional experiences — each experience must include:
       - Job title / role
       - Company name
       - Location (if available)
       - Start and end dates (if available)
       - Responsibilities or achievements (list of bullet points or sentences)
3. There can be multiple professional experiences and multiple education entries.
4. If a field is not available, fill it with a null value or an empty list, but **do not omit any field**.
5. Preserve factual accuracy and original phrasing as much as possible.
6. Return **only valid JSON** that conforms exactly to the schema, with no extra commentary or explanation.
"""

# Define a function to create a retry prompt with error feedback
def create_retry_prompt(
    original_prompt, original_response, error_message
):
    retry_prompt = f"""
This is a request to fix an error in the structure of an llm_response.
Here is the original request:
<original_prompt>
{original_prompt}
</original_prompt>

Here is the original llm_response:
<llm_response>
{original_response}
</llm_response>

This response generated an error: 
<error_message>
{error_message}
</error_message>

Compare the error message and the llm_response and identify what 
needs to be fixed or removed
in the llm_response to resolve this error. 

Respond ONLY with valid JSON. Do not include any explanations or 
other text or formatting before or after the JSON string.
"""
    return retry_prompt

class Experience(BaseModel):
    job_role: str
    responsibilities: List[str]
    environment: Optional[str]

class Resume(BaseModel):
    name: str
    phone_number: PhoneNumber
    email: EmailStr
    url: Optional[HttpUrl]
    professional_summary: List[str]
    technical_skills: List[str]
    experiences: List[Experience]

def doc_to_text(file_path):
    text = docx2txt.process(file_path)
    return text


def get_response(resume_text: str):
    start_time = timeit.default_timer()
    client = instructor.from_openai(OpenAI())
    resume_data = client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Resume,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Extract the relevant information from the following resume text:\n\n'''{resume_text}'''"}
        ],
    )
    end_time = timeit.default_timer()
    logging.info(f"Time taken: {end_time - start_time} seconds")
    return resume_data.model_dump_json(indent=2)

# Define a function to validate an LLM response
def validate_with_model(data_model, llm_response):
    try:
        validated_data = data_model.model_validate_json(llm_response)
        print("data validation successful!")
        print(validated_data.model_dump_json(indent=2))
        return validated_data, None
    except ValidationError as e:
        print(f"error validating data: {e}")
        error_message = (
            f"This response generated a validation error: {e}."
        )
        return None, error_message

# Define a function to automatically retry an LLM call multiple times
def validate_llm_response(
    prompt, data_model, n_retry=5
):
    # Initial LLM call
    response_content = get_response(prompt)
    current_prompt = prompt

    # Try to validate with the model
    # attempt: 0=initial, 1=first retry, ...
    for attempt in range(n_retry + 1):

        validated_data, validation_error = validate_with_model(
            data_model, response_content
        )

        if validation_error:
            if attempt < n_retry:
                print(f"retry {attempt} of {n_retry} failed, trying again...")
            else:
                print(f"Max retries reached. Last error: {validation_error}")
                return None, (
                    f"Max retries reached. Last error: {validation_error}"
                )

            validation_retry_prompt = create_retry_prompt(
                original_prompt=current_prompt,
                original_response=response_content,
                error_message=validation_error
            )
            response_content = get_response(
                validation_retry_prompt)
            current_prompt = validation_retry_prompt
            continue

        # If you get here, both parsing and validation succeeded
        return validated_data, None

def parse_resume(file_path):
    text_data = doc_to_text(file_path)
    resume_data = get_response(text_data)
    validated_data, validation_error  = validate_with_model(Resume, resume_data)
    while validation_error is None:
        prompt = create_retry_prompt(validation_error, )
    print(validation_error)
    prompt = create_retry_prompt(validation_error, )
    return resume_data

if __name__ == "__main__":
    load_dotenv()
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        handlers=[logging.FileHandler("parser_resume.log"), logging.StreamHandler()],
    )
    logger = logging.getLogger("parser_resume")
    file_path = "/Users/naveenpoliasetty/Downloads/RAG-1/resumes/Nithish V_Oracle consultant.docx"
    resume_data = parse_resume(file_path)

2025-11-04 21:44:18 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-04 21:45:49 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-04 21:45:49 - root - INFO - Time taken: 160.15224912500025 seconds


data validation successful!
{
  "name": "NITHISH V",
  "phone_number": "tel:+1-203-544-4365",
  "email": "nithish.v1220@gmail.com",
  "url": "https://linkedin.com/in/v-2035444365-nithish",
  "professional_summary": [
    "Results-driven Senior Oracle Consultant with 9+ years of experience delivering Oracle BRM implementations across telecom, finance, healthcare, and manufacturing, including large-scale deployments of Pipeline Manager, Pricing Design Center (PDC), and Opcode customizations.",
    "Led end-to-end design and optimization of telecom billing platforms using Oracle BRM, orchestrating high-volume Rating, multilevel Discounting, and customer lifecycle automation for Tier-1 operators.",
    "Engineered Shell Script-driven frameworks for BRM job scheduling, alerting, and crash recovery, significantly reducing operational latency across rating cycles, tax adjustments, and data reconciliation tasks.",
    "Integrated Oracle BRM with Mediation and CRM platforms using REST APIs and 

TypeError: create_retry_prompt() missing 2 required positional arguments: 'original_response' and 'error_message'

In [37]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
import time, json, re, random, requests
from typing import List, Optional
from pydantic import BaseModel


class Experience(BaseModel):
    job_role: str
    responsibilities: List[str]
    environment: Optional[str] = None


class Resume(BaseModel):
    job_role: str
    professional_summary: List[str]
    technical_skills: List[str]
    experiences: List[Experience]


class PostExtractionResult(BaseModel):
    """Result model for post extraction step."""
    job_role: Optional[str]
    structured_content: List[dict]
    full_text: str
    container_text: str
    missing_excerpt: str
    skipped_blocks: List[str]
    warnings: List[str]
    
def normalize_breaks(soup):
    """Convert <br> tags to newline text nodes so .get_text() uses them."""
    for br in soup.find_all("br"):
        br.replace_with("\n")

def clean_whitespace(text):
    lines = [ln.strip() for ln in text.splitlines()]
    while lines and lines[0] == "":
        lines.pop(0)
    while lines and lines[-1] == "":
        lines.pop()
    return "\n".join([re.sub(r'\s+', ' ', ln) for ln in lines])

def extract_job_role(soup):
    media_bodies = soup.find_all("div", class_=re.compile(r"media-body"))
    if media_bodies:
        media_body = media_bodies[0]
        job_title_tag = media_body.find("h3")
        if job_title_tag:
            job_role = job_title_tag.get_text(strip=True)
            if job_role:
                return job_role
    return None

def extract_post_body_safe(
    url: str,
    target_class: Optional[str] = None,
    class_regex: Optional[str] = None,
    allow_fallback: bool = True,
    debug: bool = False,
    min_word_threshold: int = 120,
) -> PostExtractionResult:
    """
    Extracts structured content (<p>, <ul>, <li>, .media-body job roles)
    from the main post container, skipping oversized text blocks.
    """
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"}
    resp = requests.get(url, headers=headers, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    normalize_breaks(soup)

    # Identify container
    container = None
    if target_class:
        container = soup.find("div", class_=target_class)
    if not container and class_regex:
        container = soup.find("div", class_=re.compile(class_regex))
    if not container and allow_fallback:
        divs = soup.find_all("div")
        if divs:
            container = max(divs, key=lambda d: len(d.get_text(strip=True)))
    if not container:
        raise ValueError("Could not find a suitable container.")

    raw_container_text = container.get_text(separator="\n", strip=True)
    container_text = clean_whitespace(raw_container_text)

    structured_content = []
    skipped_blocks = []  # 🔹 Track skipped text blocks

    resume_job_role = extract_job_role(soup)

    # 2️⃣ Handle normal paragraphs and lists not under media-body
    for element in container.find_all(["p", "ul"], recursive=True):
        if element.find_parent("div", class_=re.compile(r"media-body")):
            continue  # already captured above

        if element.name == "p":
            text = clean_whitespace(" ".join(element.stripped_strings))
            # 🚫 Skip overly long text blocks
            if len(text.split()) > min_word_threshold:
                skipped_blocks.append(text[:120] + "...")
                continue
            if text:
                structured_content.append({"type": "p", "text": text})

        elif element.name == "ul":
            items = []
            for li in element.find_all("li", recursive=False):
                li_text = clean_whitespace(" ".join(li.stripped_strings))
                # 🚫 Skip overly long list items
                if len(li_text.split()) > min_word_threshold:
                    skipped_blocks.append(li_text[:120] + "...")
                    continue
                if li_text:
                    items.append(li_text)
            if items:
                structured_content.append({"type": "ul", "items": items})

    # Join all paragraph text for convenience
    joined_p = "\n\n".join(
        [b["text"] for b in structured_content if b.get("type") == "p"]
    )

    container_words = len(container_text.split())
    joined_words = len(joined_p.split()) if joined_p else 0

    warnings = []
    missing_excerpt = ""
    if container_words > joined_words + 20:
        temp = container_text
        for block in structured_content:
            if block.get("type") == "p":
                temp = temp.replace(block["text"], "")
            elif block.get("type") == "ul":
                for item in block["items"]:
                    temp = temp.replace(item, "")
            elif "job_role" in block:
                temp = temp.replace(block["job_role"], "")
                for sub in block.get("content", []):
                    if sub.get("type") == "p":
                        temp = temp.replace(sub["text"], "")
                    elif sub.get("type") == "ul":
                        for item in sub["items"]:
                            temp = temp.replace(item, "")
        missing_excerpt = temp.strip()[:800]
        if missing_excerpt:
            warnings.append("Container has additional text not captured by structured tags.")

    if "<script" in resp.text.lower() and (container_words == 0 or joined_words == 0):
        warnings.append("Page might be JS-rendered.")

    if debug:
        print("===== DEBUG INFO =====")
        print("Container classes:", container.get("class"))
        print("Job roles found:", sum(1 for b in structured_content if "job_role" in b))
        print("Paragraphs:", sum(1 for b in structured_content if b.get("type") == "p"))
        print("Lists:", sum(1 for b in structured_content if b.get("type") == "ul"))
        print("Skipped blocks:", len(skipped_blocks))
        print("Warnings:", warnings)
        print("======================")

    return PostExtractionResult(
        job_role=resume_job_role,
        structured_content=structured_content,
        full_text=joined_p,
        container_text=container_text,
        missing_excerpt=missing_excerpt,
        skipped_blocks=skipped_blocks,
        warnings=warnings,
    )
url = "https://www.hireitpeople.com/resume-database/77-oracle-resumes/619638-release-and-infrastructure-management-resume-jersey-city-nj-3"
# Example usage
result = extract_post_body_safe(
    url,
    class_regex=r"(single-post-body|post-content|entry-content|article-body)",
    debug=False
)
result = result.model_dump()

# --- Preview ---
for block in result["structured_content"]:
    if "job_role" in block:
        print(f"\nJOB ROLE: {block['job_role']}")
        for sub in block["content"]:
            if sub["type"] == "p":
                print("  DESC:", sub["text"][:80])
            elif sub["type"] == "ul":
                print("  BULLETS:", sub["items"])
    elif block.get("type") == "p":
        print("PARA:", block["text"][:100])
    elif block.get("type") == "ul":
        print("LIST:", block["items"])

with open("post_content_79_2.json", "w") as f:
    f.write(json.dumps(result, indent=4, ensure_ascii=False))

PARA: SUMMARY
LIST: ['Over 9 years of working experience and expertise in implementation, Release management and Infrastructure support of Enterprise applications in J2EE environment.', 'Qualified experiencein configuration identification, data management, subversion concepts, java application deployment, release management, deployment automation.', 'Comprehensive knowledge of Software Development Life Cycle (SDLC), having thorough understanding of various phases like Requirements, Analysis, Design, Development and Testing.', 'Worked on Oracle RMB product for 3 years as Release management and Infrastructure support.', 'Experience in monitoring, troubleshooting and supporting J2EE based applications and infrastructure.', 'Proven experience of using Application Servers like Weblogic 10.3, IBM WebSphere 6.x/5.x, Tomcat 6.x/5.x/4.x.', 'Good understanding of RDBMS which includes writing sql queries using Oracle 9i/10g/11g.', 'Strong analytical and problem solving skills.', 'Experienced impl

In [38]:
result

{'job_role': 'Release And Infrastructure Management Resume',
 'structured_content': [{'type': 'p', 'text': 'SUMMARY'},
  {'type': 'ul',
   'items': ['Over 9 years of working experience and expertise in implementation, Release management and Infrastructure support of Enterprise applications in J2EE environment.',
    'Qualified experiencein configuration identification, data management, subversion concepts, java application deployment, release management, deployment automation.',
    'Comprehensive knowledge of Software Development Life Cycle (SDLC), having thorough understanding of various phases like Requirements, Analysis, Design, Development and Testing.',
    'Worked on Oracle RMB product for 3 years as Release management and Infrastructure support.',
    'Experience in monitoring, troubleshooting and supporting J2EE based applications and infrastructure.',
    'Proven experience of using Application Servers like Weblogic 10.3, IBM WebSphere 6.x/5.x, Tomcat 6.x/5.x/4.x.',
    'Good

In [39]:
def parse_resume(json_data):
    # Initialize the resume structure
    resume = {
        "job_role": json_data.get("job_role", None),
        "professional_summary": [],
        "technical_skills": [],
        "experiences": []
    }
    
    structured_content = json_data.get("structured_content", [])
    
    # Flags to track current section
    in_summary = False
    in_technical_skills = False
    in_professional_experience = False
    
    # Variables for experience parsing
    current_experience = None
    current_job_role = None
    current_responsibilities = []
    current_environment = None
    experience_started = False
    
    # Iterate through each element in structured_content
    i = 0
    while i < len(structured_content):
        element = structured_content[i]
        
        # Check if we're entering SUMMARY section
        if element["type"] == "p" and "SUMMARY" in element["text"]:
            in_summary = True
            in_technical_skills = False
            in_professional_experience = False
            i += 1
            continue
        
        # Check if we're entering TECHNICAL SKILLS section
        elif element["type"] == "p" and "TECHNICAL SKILLS" in element["text"]:
            in_summary = False
            in_technical_skills = True
            in_professional_experience = False
            i += 1
            continue
        
        # Check if we're entering PROFESSIONAL EXPERIENCE section
        elif element["type"] == "p" and "PROFESSIONAL EXPERIENCE" in element["text"]:
            in_summary = False
            in_technical_skills = False
            in_professional_experience = True
            i += 1
            continue
        
        # Process SUMMARY section
        elif in_summary:
            if element["type"] == "ul":
                resume["professional_summary"].extend(element["items"])
            i += 1
            continue
        
        # Process TECHNICAL SKILLS section
        elif in_technical_skills:
            if element["type"] == "p":
                # Skip the "TECHNICAL SKILLS:" header itself
                if "TECHNICAL SKILLS" not in element["text"]:
                    resume["technical_skills"].append(element["text"])
            i += 1
            continue
        
        # Process PROFESSIONAL EXPERIENCE section
        elif in_professional_experience:
            # Check for "Confidential" P tags to identify new experiences
            if element["type"] == "p" and "Confidential" in element["text"]:
                # Save previous experience if it exists and has data
                if experience_started and current_job_role:
                    experience_data = {
                        "job_role": current_job_role,
                        "responsibilities": current_responsibilities.copy()
                    }
                    if current_environment:
                        experience_data["environment"] = current_environment
                    resume["experiences"].append(experience_data)
                
                # Reset for new experience
                current_job_role = None
                current_responsibilities = []
                current_environment = None
                experience_started = True
                
                # The next P tag after "Confidential" should be the job role
                if i + 1 < len(structured_content):
                    next_element = structured_content[i + 1]
                    if next_element["type"] == "p":
                        current_job_role = next_element["text"]
                        i += 2  # Skip both confidential and job role
                    else:
                        i += 1
                else:
                    i += 1
                continue
            
            # Check for UL tags (responsibilities)
            elif element["type"] == "ul" and current_job_role:
                current_responsibilities.extend(element["items"])
                i += 1
                continue
            
            # Check for "Environment" P tags
            elif element["type"] == "p" and "Environment:" in element["text"]:
                current_environment = element["text"].replace("Environment:", "").strip()
                i += 1
                continue
            
            # Check for "Environment" without colon (some entries might have different formatting)
            elif element["type"] == "p" and "Environment" in element["text"] and current_job_role:
                current_environment = element["text"].replace("Environment", "").strip()
                if current_environment.startswith(":"):
                    current_environment = current_environment[1:].strip()
                i += 1
                continue
            
            # Regular P tag in experience section (might be environment or other info)
            elif element["type"] == "p":
                # If we don't have a job role yet but we're in an experience, this might be it
                if not current_job_role and "Confidential" not in element["text"] and "PROFESSIONAL EXPERIENCE" not in element["text"]:
                    current_job_role = element["text"]
                i += 1
                continue
            
            else:
                i += 1
                continue
        
        else:
            i += 1
            continue
    
    # Don't forget to add the last experience if it exists
    if experience_started and current_job_role:
        experience_data = {
            "job_role": current_job_role,
            "responsibilities": current_responsibilities.copy()
        }
        if current_environment:
            experience_data["environment"] = current_environment
        resume["experiences"].append(experience_data)
    
    return resume

# Alternative version that returns a Resume object
def parse_resume_to_object(json_data):
    parsed_data = parse_resume(json_data)
    return Resume(**parsed_data)

# Usage example:
if __name__ == "__main__":
    # Assuming your JSON data is in a variable called 'sample_json'
    resume_data = parse_resume(result)
    print(type(resume_data))
    #print(resume_data.model_dump_json(indent=4))
    with open("lalalal.json", "w") as f:
        json.dump(resume_data, f, indent=4)
    


<class 'dict'>
